### Assignment 3 - Data Stewardship

## Deadline:  December 12, 23:59  [y.he2@uva.nl](mailto:y.he2@uva.nl)

## Upload your notebook file on Canvas

|**Name** | **Student id** | **email**|
|:- |:-|:-|
|narendra parigi | 12089265 | naren.parigi@gmail.com|

I (enlisted above) declare that:
1. My assignment will be my own work.
2. I shall not make solutions to the assignment available to anyone else.
3. I shall not engage in any other activities that will dishonestly improve my results or dishonestly improve or hurt the results of others.

Note:
- your notebook should give no errors when executed as "Run All" cells
- ** Use markdown cell setting for comments/additional explanation **

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import NaN
import scipy.stats as stats
from pandas import Series, DataFrame
import pandas as pd
plt.rc('figure', figsize=(10, 6))
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

# <font color='blue'>Question 1 [7 points] - Cleaning the scraped Nobel Prize dataset</font>

The first part of this assignment consists of finding and eliminating dirty data in the Nobel Prize dataset. The final aim is to have a cleaned dataset that is ready to be explored by EDA.
The following points can be considered:  
   
1. The first Nobel was awarded in 1901. Remove the earlier records.  

2. Replace empty strings, i.e. <tt>''</tt>, with <tt>NaN</tt>

3. Some names are marked with an asterisk, denoting that these winners are recorded by country of birth, not country at the time of winning the prize. Clean up those names by removing the asterisks and strip any remaining whitespace. You may need to input ```'\*'``` for an asterisk string if you use the **contains** method. Generate a flag variable that equals 1 if the country is the country of birth.

4. Some names are duplicated. This could indicate that some people have won the Nobel Prize more than once. However, it could also be that a winner is "claimed" by more than one country. Remove all real duplicates. For instance, Marie Curie is contained 4 times: twice for Poland and twice for France. Remove 2 of them (you have to make a choice here: remove winners country alphabetically of not?)

5. Consider the missing values in the field <font color='blue'>category</font>. Are they really missing or just entry mistakes? [You don't have to complete missing values]  

6. Is there an explanation of the missing values of the field <font color='blue'>gender</font>? Remove all records without gender.

7. What about missing values for <font color='blue'>date_of_birth</font>?  

Note: all changes should be made within this notebook. So, *no* alterations may be made to the input file outside this notebook (e.g. by using a file editor). When you have a choice, please <font color='red'>motivate</font>. *Show that you are aware of the consequences of your decision* (sometimes there is no right/wrong).

### Read Data

In [8]:
df_nobel = pd.read_json("nobel_winners.json")
df_ratings = pd.read_csv("JudgeRatings.csv")

In [9]:
df_nobel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 12 columns):
born_in           1052 non-null object
category          1052 non-null object
country           1052 non-null object
date_of_birth     1044 non-null object
date_of_death     1044 non-null object
gender            1040 non-null object
link              1052 non-null object
name              1052 non-null object
place_of_birth    1044 non-null object
place_of_death    1044 non-null object
text              1052 non-null object
year              1052 non-null int64
dtypes: int64(1), object(11)
memory usage: 98.7+ KB


In [10]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 13 columns):
JUDGE    43 non-null object
CONT     43 non-null float64
INTG     43 non-null float64
DMNR     43 non-null float64
DILG     43 non-null float64
CFMG     43 non-null float64
DECI     43 non-null float64
PREP     43 non-null float64
FAMI     43 non-null float64
ORAL     43 non-null float64
WRIT     43 non-null float64
PHYS     43 non-null float64
RTEN     43 non-null float64
dtypes: float64(12), object(1)
memory usage: 4.4+ KB


In [11]:
(df_nobel.shape, df_ratings.shape)

((1052, 12), (43, 13))

### Question 1

In [13]:
df_nobel_fil = df_nobel[df_nobel['year'] >= 1901]

In [14]:
df_nobel_fil.sort_values(by=['year']).head(2)

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year
67,,Physics,Germany,27 March 1845,10 February 1923,male,http://en.wikipedia.org/wiki/Wilhelm_Conrad_R%...,Wilhelm Conrad Röntgen,"Lennep , Prussia","Munich , Germany","Wilhelm Conrad Röntgen , Physics, 1901",1901
909,,Peace,France,8 May 1828,30 October 1910,male,http://en.wikipedia.org/wiki/Henry_Dunant,Henry Dunant,"Geneva , Switzerland","Heiden, Switzerland","Henry Dunant , Peace, 1901",1901


In [15]:
print ('The size of the nobel dataframe after filtering records {0}'.format(df_nobel_fil.shape[0]))

The size of the nobel dataframe after filtering records 1049


### Question 2

In [16]:
print("The following columns have have an empty string as value")
for column in df_nobel_fil.columns:
    if (df_nobel_fil[column] == '').any():
        print(column)

The following columns have have an empty string as value
born_in
category
country
date_of_birth
date_of_death
place_of_birth
place_of_death


In [17]:
df_nobel_fil.replace('', np.nan, inplace=True)

In [18]:
print("The following columns have have an empty string as value")
for column in df_nobel_fil.columns:
    if (df_nobel_fil[column] == '').any():
        print(column)

The following columns have have an empty string as value


All columns with empty string as value are replaced by np.nan

### Question 3

In [22]:
df_nobel_fil[df_nobel_fil['name'].str.contains('\*', regex=True)==True].shape

(142, 13)

In [20]:
df_nobel_fil['flag_country_check'] = df_nobel_fil['name'].str.contains('\*', regex=True).astype(int)

In [21]:
df_nobel_fil['flag_country_check'].value_counts()

0    907
1    142
Name: flag_country_check, dtype: int64

In total 142 entries contain a name with a asterisks (*) and a flag is generated to represent a section.

### Question 4

In [23]:
df_nobel_fil[df_nobel_fil['name'].str.contains('Curie') & df_nobel_fil['name'].str.contains('Marie')]

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,flag_country_check
706,NaN,Physics,Poland,7 November 1867,4 July 1934,female,http://en.wikipedia.org/wiki/Marie_Curie,Marie Skłodowska-Curie,"Warsaw , Poland","Sancellemoz , France","Marie Skłodowska-Curie , born in partitioned ...",1903,0
709,NaN,Chemistry,Poland,7 November 1867,4 July 1934,female,http://en.wikipedia.org/wiki/Marie_Curie,Marie Skłodowska-Curie,"Warsaw , Poland","Sancellemoz , France","Marie Skłodowska-Curie , born in partitioned ...",1911,0
910,NaN,Physics,France,7 November 1867,4 July 1934,female,http://en.wikipedia.org/wiki/Marie_Curie,Marie Curie,"Warsaw , Poland","Sancellemoz , France","Marie Curie , born in then Russian Empire, no...",1903,0
919,NaN,Chemistry,France,7 November 1867,4 July 1934,female,http://en.wikipedia.org/wiki/Marie_Curie,Marie Curie,"Warsaw , Poland","Sancellemoz , France","Marie Curie , born in then Russian Empire, no...",1911,0


In [24]:
df_nobel_fil_dedup = df_nobel_fil.sort_values(by=['country']).drop_duplicates(
    subset=['category', 'date_of_birth', 'gender','year'], 
    keep='first')

In [25]:
df_nobel_fil_dedup.shape

(874, 13)

In [26]:
df_nobel_fil_dedup[df_nobel_fil_dedup['name'].str.contains('Curie') &
                   df_nobel_fil_dedup['name'].str.contains('Marie')]

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,flag_country_check
910,NaN,Physics,France,7 November 1867,4 July 1934,female,http://en.wikipedia.org/wiki/Marie_Curie,Marie Curie,"Warsaw , Poland","Sancellemoz , France","Marie Curie , born in then Russian Empire, no...",1903,0
919,NaN,Chemistry,France,7 November 1867,4 July 1934,female,http://en.wikipedia.org/wiki/Marie_Curie,Marie Curie,"Warsaw , Poland","Sancellemoz , France","Marie Curie , born in then Russian Empire, no...",1911,0


The sub-set of columns category, date of birth, gender and year used to exclude duplicates as the category, date of birth, gender and year combination of columns must be identical, although the country and name differ in the entries.

Note: we can also use date_of_death, place_of_birth, place_of_death as sub-set columns but the combination category, date of birth, gender and year is already unique. 

### Question 5

In [27]:
df_nobel_fil_dedup['category'].value_counts()

Physiology or Medicine    206
Physics                   200
Chemistry                 169
Literature                110
Peace                     109
Economics                  74
Name: category, dtype: int64

In [28]:
df_nobel_fil_dedup[df_nobel_fil_dedup['category'].isna() | df_nobel_fil_dedup['category'].isnull()]

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,flag_country_check
922,NaN,NaN,France,28 June 1873,5 November 1944,male,http://en.wikipedia.org/wiki/Alexis_Carrel,Alexis Carrel,"Sainte-Foy-lès-Lyon , Rhône , France",NaN,"Alexis Carrel , Medicine, 1912",1912,0
104,Lithuania,NaN,NaN,"November 30, 1926",NaN,male,http://en.wikipedia.org/wiki/Andrew_Schally,Andrew Schally *,"Vilnius , Second Polish Republic",NaN,"Andrew Schally *, born in then Poland, now Li...",1977,1
712,Poland,NaN,NaN,"November 30, 1926",NaN,male,http://en.wikipedia.org/wiki/Andrew_Schally,Andrew Schally *,"Vilnius , Second Polish Republic",NaN,"Andrew Schally *, born in Wilno, Second Polis...",1976,1
812,India,NaN,NaN,"January 9, 1922","November 9, 2011",male,http://en.wikipedia.org/wiki/Hargobind_Khorana,Hargobind Khorana *,"Raipur , Punjab , Pakistan","Concord, Massachusetts , U.S.","Hargobind Khorana *, born in Kabirwala , Bri...",1968,1
815,India,NaN,NaN,3 November 1933,NaN,male,http://en.wikipedia.org/wiki/Amartya_Sen,Amartya Sen *,"Dhaka , Bangladesh",NaN,"Amartya Sen *, Economic Sciences, 1998",1998,1
820,Hungary,NaN,NaN,22 April 1876,8 April 1936,male,http://en.wikipedia.org/wiki/Robert_B%C3%A1r%C...,Robert Bárány *,"Vienna , Austria-Hungary","Uppsala , Sweden","Robert Bárány *, born in then Austria-Hungary...",1914,1


The six entries in the column of category are filled with nan values. The actual values in the json file are empty strings and have been replaced by np.nan as part of the question (2). This is certainly a data entry error, as, for example, Amartya Sen won the Nobel Prize in Economics.

### Question 6

In [29]:
df_nobel_fil_dedup['gender'].value_counts()

male      818
female     47
Name: gender, dtype: int64

In [33]:
df_nobel_fil_dedup[df_nobel_fil_dedup['gender'].isnull() | df_nobel_fil_dedup['gender'].isna()]

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,flag_country_check
1033,NaN,Peace,Austria,None,None,None,http://en.wikipedia.org/wiki/International_Ato...,International Atomic Energy Agency,None,None,"International Atomic Energy Agency , Peace, 2005",2005,0
3,NaN,Peace,Belgium,None,None,None,http://en.wikipedia.org/wiki/Institut_de_Droit...,Institut de Droit International,None,None,"Institut de Droit International , Peace, 1904",1904,0
1000,NaN,Peace,Canada,None,None,None,http://en.wikipedia.org/wiki/Pugwash_Conferenc...,Pugwash Conferences on Science and World Affairs,None,None,Pugwash Conferences on Science and World Affai...,1995,0
947,NaN,Peace,France,None,None,None,http://en.wikipedia.org/wiki/M%C3%A9decins_San...,Médecins Sans Frontières,None,None,"Médecins Sans Frontières , Peace, 1999",1999,0
650,NaN,Physiology or Medicine,Sweden,"October 30, 1900","March 12, 1991",None,http://en.wikipedia.org/wiki/Ragnar_Granit,Ragnar Granit,"Helsinki , Finland","Stockholm , Sweden","Ragnar Granit , born in then Russian Empire, ...",1967,0
574,NaN,Peace,United Kingdom,None,None,None,http://en.wikipedia.org/wiki/Amnesty_Internati...,Amnesty International,None,None,"Amnesty International , Peace, 1977",1977,0
156,NaN,Peace,United Kingdom,None,None,None,http://en.wikipedia.org/wiki/Friends_Service_C...,Friends Service Council,None,None,"Friends Service Council , Peace, 1947",1947,0
696,Russia,Physics,NaN,23 August 1974,NaN,None,http://en.wikipedia.org/wiki/Konstantin_Novoselov,Konstantin Novoselov *,"Nizhny Tagil , Russian SFSR , Soviet Union",NaN,"Konstantin Novoselov *, Physics, 2010",2010,1
989,Canada,Physiology or Medicine,NaN,"September 22, 1901","January 12, 1997",None,http://en.wikipedia.org/wiki/Charles_Brenton_H...,Charles B. Huggins *,"Halifax, Nova Scotia","Chicago, Illinois","Charles B. Huggins *, Physiology or Medicine, ...",1966,1


In [34]:
df_nobel_fil_dedup.dropna(subset=['gender'], inplace=True)
df_nobel_fil_dedup.shape

(865, 13)

There are 9 gender incomplete entries. 6 out of 9 entries have no gender as they serve non-personal institutes, but 3 out of 9 are gender missing, may be data entry problem.

### Question7 

In [35]:
df_nobel_fil_dedup[df_nobel_fil_dedup['date_of_birth'].isnull() | df_nobel_fil_dedup['date_of_birth'].isna()]

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,flag_country_check
782,NaN,Physics,Japan,NaN,NaN,male,http://en.wikipedia.org/wiki/Hiroshi_Amano,Hiroshi Amano,NaN,NaN,"Hiroshi Amano , Physics, 2014",2014,0


There is 1 entry with missing date_of_birth. This could also be a problem with data entry.

# <font color='blue'>Question 2 [3 points] - US Judge Ratings</font>

In the file <tt>JudgeRatings.csv</tt>, you can find the information about Lawyers' ratings of state judges in the US Superior Court. The file contains 43 observations on 12 numeric variables.

- CONT 	Number of contacts of lawyer with judge.
- INTG 	Judicial integrity.
- DMNR 	Demeanor.
- DILG 	Diligence.
- CFMG 	Case flow managing.
- DECI 	Prompt decisions.
- PREP 	Preparation for trial.
- FAMI 	Familiarity with law.
- ORAL 	Sound oral rulings.
- WRIT 	Sound written rulings.
- PHYS 	Physical ability.
- RTEN 	Worthy of retention.


1. Apply PCA to the 12 components after standardizing. Determine how many components to use based on (motivate you answers):
 - Eigenvalue Criterion
 - Proportion of Variance Explained Criterion (use a minimum of 90% variability)
  
  
2. Display the component matrix, that is, the correlation matrix between the features and the components. Round off the values to 2 decimal places. 

3. Comment on the above results.